In [1]:
from prodock.io.pdb_query import PDBQuery
proc = PDBQuery("5N2F", "Data/testcase", chains=["A", "B"], ligand_code="8HW")
proc.run_all()
print("raw:", proc.cocrystal_ligand_path)
print("processed:", proc.reference_ligand_path)

raw: /homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/cocrystal/5N2F.sdf
processed: /homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/reference_ligand/8HW.sdf


1 molecule converted
1 molecule converted


In [2]:
from prodock.process.gridbox import GridBox
from prodock.vis.provis import ProVis
from prodock.vis.provis_gui import ProVisGUI


In [3]:
gb = GridBox().load_ligand("Data/testcase/cocrystal/5N2F.sdf", fmt="sdf") \
              .from_ligand_pad(pad=4.0, isotropic=True)

print("Box dict for Vina:", gb.vina_dict)
print("Box config snippet:\n", gb.to_vina_lines())


Box dict for Vina: {'center_x': 32.536, 'center_y': 12.991, 'center_z': 133.858, 'size_x': 23.594, 'size_y': 23.594, 'size_z': 23.594}
Box config snippet:
 center_x = 32.536
center_y = 12.991
center_z = 133.858
size_x = 23.594
size_y = 23.594
size_z = 23.594


In [4]:
viz = ProVis(800, 600)
viz.load_receptor("Data/testcase/filtered_protein/5N2F.pdb") \
   .set_receptor_style("cartoon", "white") \
   .load_ligand("Data/testcase/reference_ligand/8HW.sdf", fmt="sdf") \
   .highlight_ligand(style="stick", color="cyan") \
   .add_gridbox_from(gb, labels=True) \
   .show()


3Dmol.js failed to load for some reason. Please check your browser console for error messages.

<ProVis models=2, ligands=1>

In [5]:
gui = ProVisGUI().build().display()


In [7]:
from prodock.process.target import ProteinProcess

pp = ProteinProcess(enable_logging=True)
pp.fix_and_minimize_pdb(
    input_pdb="Data/testcase/filtered_protein/5N2F.pdb",
    output_dir="Data/testcase/filtered_protein/",
    energy_diff=10.0,
    max_minimization_steps=5000,
    pdb_id="5N2F",
    protein_name="5N2F",
    minimize_in_water=False,
    input_ligand='Data/testcase/reference_ligand/8HW.sdf',
    out_fmt="gpf",   # default; will produce Data/testcase/filtered_protein/5N2F_min.pdb
)

# print("Artifact:", pp.final_pdb_path)
print("Report:", pp.last_simulation_report)


2025-08-27 15:57:03,304 - INFO - ProteinProcess: fixing Data/testcase/filtered_protein/5N2F.pdb -> Data/testcase/filtered_protein (out_fmt=gpf)
2025-08-27 15:57:04,263 - DEBUG - OpenMM platform selected: CPU
2025-08-27 15:57:04,304 - INFO - Minimizing (gas phase)... tol=10.0 maxIter=5000
2025-08-27 15:57:17,548 - DEBUG - GridBox.preset('safe') succeeded
2025-08-27 15:57:17,549 - DEBUG - Calling mekoo: /homes/biertank/tieu/miniconda3/envs/prodock/bin/mk_prepare_receptor.py --read_pdb Data/testcase/filtered_protein/5N2F.pdb -o Data/testcase/filtered_protein/5N2F --box_center 32.5 13.0 133.75 --box_size 23.5 23.5 23.5 --write_pdbqt Data/testcase/filtered_protein/5N2F.pdbqt --write_gpf Data/testcase/filtered_protein/5N2F.gpf
2025-08-27 15:57:22,483 - DEBUG - mekoo rc=0 stdout_len=406 stderr_len=0
2025-08-27 15:57:22,486 - DEBUG - mekoo produced: ['/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/filtered_protein/5N2F.pdbqt', '/homes/biertank/tieu/Documents/Project/

Report: {'final_artifact': '/homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/filtered_protein/5N2F.gpf', 'out_fmt': 'gpf', 'mekoo_info': {'called': '/homes/biertank/tieu/miniconda3/envs/prodock/bin/mk_prepare_receptor.py --read_pdb Data/testcase/filtered_protein/5N2F.pdb -o Data/testcase/filtered_protein/5N2F --box_center 32.5 13.0 133.75 --box_size 23.5 23.5 23.5 --write_pdbqt Data/testcase/filtered_protein/5N2F.pdbqt --write_gpf Data/testcase/filtered_protein/5N2F.gpf', 'rc': 0, 'stdout': '\nFiles written:\n                Data/testcase/filtered_protein/5N2F.pdbqt <-- static (i.e., rigid) receptor input file\nData/testcase/filtered_protein/boron-silicon-atom_par.dat <-- atomic parameters for B and Si (for autogrid)\n                  Data/testcase/filtered_protein/5N2F.gpf <-- autogrid input file\n              Data/testcase/filtered_protein/5N2F.box.pdb <-- PDB file to visualize the grid box\n', 'stderr': '', 'produced': ['/homes/biertank/tieu/Documents/Proj

In [8]:
from prodock.io.convert import Converter
conv = Converter().set_input("Data/testcase/reference_ligand/8HW.sdf").set_output("Data/testcase/reference_ligand/8HW.pdbqt").set_mode("ligand").use_meeko().run()
print("output:", conv.output)
print("requested mode:", conv._mode)        # 'ligand'
print("preferred backend:", conv._prefer)  # 'meeko' (because you called use_meeko())


2025-08-27 15:57:23 | WARNING | prodock.io.convert | Meeko failed: /homes/biertank/tieu/miniconda3/envs/prodock/bin/mk_prepare_ligand.py failed (rc=1)
STDOUT:
*ERROR* Format [pdb] not in supported formats [sdf/mol2/mol]

STDERR:



output: /homes/biertank/tieu/Documents/Project/TACsy/ProEco/ProDock/Data/testcase/reference_ligand/8HW.pdbqt
requested mode: ligand
preferred backend: meeko


In [9]:
from prodock.engine.vina import VinaDock

In [10]:
vd = (VinaDock(sf_name="vina", cpu=4, seed=42)
      .set_receptor("Data/testcase/filtered_protein/5N2F.pdbqt")
      .define_box(center=(32.5, 13.0, 133.75), size=(22.5, 23.5, 22.5))
)

# single ligand
vd.set_ligand("Data/testcase/reference_ligand/8HW.pdbqt").dock(exhaustiveness=8, n_poses=9).write_poses("./Data/testcase/vina/vina_out.pdbqt").write_log("./Data/testcase/vina/log.txt")
print("scores:", vd.scores)
print("best:", vd.get_best())


Computing Vina grid ... done.
Performing docking (random seed: 42) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal/mol) | rmsd l.b.| rmsd u.b.
-----+------------+----------+----------
   1       -11.02          0          0
   2       -11.01      2.026      2.768
   3       -10.91      2.579       3.82
   4       -10.76       1.91      2.136
   5       -10.15      1.295      1.619
   6        -9.93      3.323      12.02
   7        -9.87      3.422      11.25
   8       -9.607      3.601      11.57
   9       -9.469      2.968      12.07
Computing Vina grid ... done.
Performing docking (random seed: auto) ... 
0%   10   20   30   40   50   60   70   80   90   100%
|----|----|----|----|----|----|----|----|----|----|
***************************************************

mode |   affinity | dist from best mode
     | (kcal

In [11]:
from prodock.engine.binary import BinaryDock

In [12]:
bd = (
    BinaryDock("qvina-w")
    .set_receptor("5N2F.pdbqt")
    .set_ligand("8HW.pdbqt")
    .set_out("docked/8HW_docked.pdbqt")
    .set_log("docked/8HW.log")
    .set_box( center=(32.500, 13.0, 133.750), size=(22.500, 23.500, 22.500))
    # .enable_autobox("Data/testcase/reference_ligand/8HW.sdf", padding=4.0)
    .set_exhaustiveness(2)
    .set_num_modes(9)
    .set_cpu(4)
    .set_seed(42)
    .run()
)
# parse rows
rows = bd.parse_scores_from_log("docked/8HW.log")
print(rows)
# write CSV
bd.scores_to_csv("docked/8HW.log", "docked/8HW_scores.csv")
# get pandas DataFrame (requires pandas)
df = bd.scores_as_dataframe("docked/8HW.log")
print(df)


FileNotFoundError: Missing input file: 5N2F.pdbqt

In [ ]:
from prodock.io.rdkit_io import smiles2sdf
smiles2sdf("c1ccccc1", "test.sdf", embed3d=True, add_hs=True, optimize=True)

In [ ]:
from prodock.process.ligand import LigandProcess 

rows = [{"smiles": "CCO", "name": "ethanol"},
       {"smiles": "c1ccccc1", "name": "benzene"}]

lp = LigandProcess(output_dir='out', name_key="name")
lp.from_list_of_dicts(rows)
lp.set_embed_method("ETKDGv3").set_opt_method("MMFF94s")
lp.process_all()